<a href="https://colab.research.google.com/github/Bright-Momo/SimplifiedJournalsGame/blob/main/3%20author%20games%20no%20nashpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import itertools
from tqdm import tqdm

In [3]:
# Parameters (same as in the original code)
alpha = 0.6    # Probability of a good paper
epsilon = 0.1  # Probability of misclassifying a good paper as bad
lam = 0.1      # Probability of misclassifying a bad paper as good
r = 1.0        # Reward for author when accepted
c = 0.5        # Cost for author when rejected

strategies = [0, 1, 2, 3]  # Always Submit, Only Submit Good, Only Submit Bad, No Submit

In [4]:
# Function to calculate acceptance probability
def acceptance_probability(is_good_paper, journal_strategy, epsilon, lam):
    if journal_strategy == 0:  # Always Accept
        return 1.0
    elif journal_strategy == 1:  # Only Accept Good
        return 1 - epsilon if is_good_paper else lam
    elif journal_strategy == 2:  # Only Accept Bad
        return epsilon if is_good_paper else 1 - lam
    else:  # Reject All
        return 0.0

In [6]:
# Calculate author and journal payoffs
def author_journal_payoff(author_strategy, journal_strategy, alpha, epsilon, lam, r, c):
    if author_strategy == 0:  # Always Submit
        good_prob, bad_prob = alpha, 1 - alpha
    elif author_strategy == 1:  # Only Submit Good
        good_prob, bad_prob = alpha, 0
    elif author_strategy == 2:  # Only Submit Bad
        good_prob, bad_prob = 0, 1 - alpha
    else:  # No Submit
        good_prob, bad_prob = 0, 0

    p_accept_good = acceptance_probability(True, journal_strategy, epsilon, lam)
    p_accept_bad = acceptance_probability(False, journal_strategy, epsilon, lam)

    accept_rate = good_prob * p_accept_good + bad_prob * p_accept_bad
    submit_rate = good_prob + bad_prob
    reject_rate = submit_rate - accept_rate

    author_payoff = accept_rate * r - reject_rate * c

    # Average quality of accepted papers (journal's payoff)
    if accept_rate > 0:
        journal_payoff = (good_prob * p_accept_good) / accept_rate
    else:
        journal_payoff = 0

    return author_payoff, journal_payoff

In [7]:
# Calculate author payoffs for a given strategy profile
def calculate_author_payoffs(author_strategies, journal_strategy, alpha, epsilon, lam, r, c):
    return [author_journal_payoff(author_strategy, journal_strategy, alpha, epsilon, lam, r, c)[0]
            for author_strategy in author_strategies]

# Calculate journal payoff for a given strategy profile
def calculate_journal_payoff(author_strategies, journal_strategy, alpha, epsilon, lam, r, c):
    journal_payoffs = [author_journal_payoff(author_strategy, journal_strategy, alpha, epsilon, lam, r, c)[1]
                      for author_strategy in author_strategies]
    # Average quality of all accepted papers
    return np.mean(journal_payoffs)


In [8]:
# Check if a strategy profile is a Nash equilibrium
def is_nash_equilibrium(strategy_profile, alpha, epsilon, lam, r, c):
    author_strategies = strategy_profile[:-1]  # First 3 elements
    journal_strategy = strategy_profile[-1]    # Last element

    # Calculate current payoffs
    author_payoffs = calculate_author_payoffs(author_strategies, journal_strategy, alpha, epsilon, lam, r, c)
    journal_payoff = calculate_journal_payoff(author_strategies, journal_strategy, alpha, epsilon, lam, r, c)

    # Check if any author can improve by deviating
    for i in range(len(author_strategies)):
        for alt_strategy in strategies:
            if alt_strategy == author_strategies[i]:
                continue

            # Create new strategy profile with one author deviating
            new_author_strategies = list(author_strategies)
            new_author_strategies[i] = alt_strategy

            # Calculate payoff with deviation
            new_payoff = author_journal_payoff(alt_strategy, journal_strategy, alpha, epsilon, lam, r, c)[0]

            # If author can improve by deviating, this is not a Nash equilibrium
            if new_payoff > author_payoffs[i]:
                return False

    # Check if journal can improve by deviating
    for alt_strategy in strategies:
        if alt_strategy == journal_strategy:
            continue

        # Calculate journal payoff with deviation
        new_journal_payoff = calculate_journal_payoff(author_strategies, alt_strategy, alpha, epsilon, lam, r, c)

        # If journal can improve by deviating, this is not a Nash equilibrium
        if new_journal_payoff > journal_payoff:
            return False

    # If no player can improve by deviating, this is a Nash equilibrium
    return True

In [9]:
# Find all Nash equilibria
def find_nash_equilibria(alpha, epsilon, lam, r, c):
    nash_equilibria = []

    # Generate all possible strategy profiles
    all_strategy_profiles = list(itertools.product(strategies, repeat=4))  # 3 authors + 1 journal

    print(f"Checking {len(all_strategy_profiles)} strategy profiles for Nash equilibria...")

    # Check each strategy profile
    for profile in tqdm(all_strategy_profiles):
        if is_nash_equilibrium(profile, alpha, epsilon, lam, r, c):
            nash_equilibria.append(profile)

    return nash_equilibria


In [10]:
# Display results in a more readable format
def display_equilibria(equilibria, alpha, epsilon, lam, r, c):
    print(f"\nFound {len(equilibria)} Nash equilibria:")

    for i, eq in enumerate(equilibria):
        author_strategies = eq[:-1]
        journal_strategy = eq[-1]

        author_payoffs = calculate_author_payoffs(author_strategies, journal_strategy, alpha, epsilon, lam, r, c)
        journal_payoff = calculate_journal_payoff(author_strategies, journal_strategy, alpha, epsilon, lam, r, c)

        print(f"\nEquilibrium #{i+1}:")
        print(f"  Author strategies: {author_strategies}")
        print(f"  Journal strategy: {journal_strategy}")
        print(f"  Author payoffs: {author_payoffs}")
        print(f"  Journal avg quality: {journal_payoff:.2f}")

        # Interpret strategies
        author_strat_names = []
        for s in author_strategies:
            if s == 0:
                author_strat_names.append("Always Submit")
            elif s == 1:
                author_strat_names.append("Only Submit Good")
            elif s == 2:
                author_strat_names.append("Only Submit Bad")
            else:
                author_strat_names.append("No Submit")

        if journal_strategy == 0:
            journal_strat_name = "Always Accept"
        elif journal_strategy == 1:
            journal_strat_name = "Only Accept Good"
        elif journal_strategy == 2:
            journal_strat_name = "Only Accept Bad"
        else:
            journal_strat_name = "Reject All"

        print(f"  Interpretation:")
        for i, strat in enumerate(author_strat_names):
            print(f"    Author {i+1}: {strat}")
        print(f"    Journal: {journal_strat_name}")

In [11]:
# Run the analysis
if __name__ == "__main__":
    # Get Nash equilibria
    equilibria = find_nash_equilibria(alpha, epsilon, lam, r, c)

    # Display results
    display_equilibria(equilibria, alpha, epsilon, lam, r, c)

Checking 256 strategy profiles for Nash equilibria...


100%|██████████| 256/256 [00:00<00:00, 35569.68it/s]


Found 3 Nash equilibria:

Equilibrium #1:
  Author strategies: (1, 1, 1)
  Journal strategy: 1
  Author payoffs: [0.51, 0.51, 0.51]
  Journal avg quality: 1.00
  Interpretation:
    Author 1: Only Submit Good
    Author 2: Only Submit Good
    Author 3: Only Submit Good
    Journal: Only Accept Good

Equilibrium #2:
  Author strategies: (2, 2, 2)
  Journal strategy: 2
  Author payoffs: [0.3400000000000001, 0.3400000000000001, 0.3400000000000001]
  Journal avg quality: 0.00
  Interpretation:
    Author 1: Only Submit Bad
    Author 2: Only Submit Bad
    Author 3: Only Submit Bad
    Journal: Only Accept Bad

Equilibrium #3:
  Author strategies: (3, 3, 3)
  Journal strategy: 3
  Author payoffs: [0.0, 0.0, 0.0]
  Journal avg quality: 0.00
  Interpretation:
    Author 1: No Submit
    Author 2: No Submit
    Author 3: No Submit
    Journal: Reject All
